<a href="https://colab.research.google.com/github/Cuka3077/CSCI6366_Neural-Networks-and-Deep-Learning/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task 1: Image Classification with CNN (Initial Stage)


In [42]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

Load the Fashion-MNIST train and test CSV files, normalize pixel values, reshape them into image tensors, and build DataLoaders

In [43]:
# 1. Load dataset & basic preprocessing
TRAIN_CSV_PATH = "/content/fashion-mnist_train.csv"
TEST_CSV_PATH  = "/content/fashion-mnist_test.csv"

# Load the training and test splits
train_df = pd.read_csv(TRAIN_CSV_PATH)
test_df  = pd.read_csv(TEST_CSV_PATH)

def preprocess_fashion_df(df):
    # Labels: class ids from 0 to 9
    y = df.iloc[:, 0].values

    # Pixel data, normalized to [0, 1]
    X = df.iloc[:, 1:].values / 255.0

    # Reshape to image tensors of shape N x 1 x 28 x 28
    X = X.reshape(-1, 1, 28, 28)

    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.long)
    return X, y

# Preprocess both train and test splits
X_train, y_train = preprocess_fashion_df(train_df)
X_test,  y_test  = preprocess_fashion_df(test_df)

# Build DataLoaders: shuffle for training, no shuffle for test
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
val_loader   = DataLoader(TensorDataset(X_test,  y_test),  batch_size=64, shuffle=False)

Define a simple CNN model for Fashion-MNIST without inserting any nonlinear activation functions between layers, serving as the linear baseline.

In [44]:
# 2. Define CNN model (no activations yet)
class FashionCNNLinear(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # Convolution + pooling (no activation here)
        x = self.pool1(self.conv1(x))
        x = self.pool2(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

Configure the device, loss function, optimizer, and run a minimal training loop (1 epoch) to verify that the CNN pipeline works end-to-end on train/test.

In [45]:
# 3. Training loop (minimal, 1 epoch)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FashionCNNLinear().to(device)

# Use cross-entropy loss for multi-class classification
criterion = nn.CrossEntropyLoss()

# Use Adam optimizer with a learning rate of 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(1):
    # Training phase
    model.train()
    for Xb, yb in train_loader:
        Xb, yb = Xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits = model(Xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

    # Evaluation phase
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for Xb, yb in val_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            logits = model(Xb)
            preds = logits.argmax(dim=1)
            correct += (preds == yb).sum().item()
            total += yb.size(0)
    val_acc = correct / total
    print(f"Epoch 0, val_acc={val_acc:.4f}")

Epoch 0, val_acc=0.8830


Task 2: Loan Default + MLP

In [46]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Load the loan payments dataset and create the binary target column 'loan_status_binary' where 0 = PAIDOFF and 1 = all other statuses.

In [47]:
# 1. Load dataset & create binary label

CSV_PATH = "/content/Loan payments data.csv"

df = pd.read_csv(CSV_PATH)

# Create a binary classification label:
#   PAIDOFF      -> 0 (no default)
#   others       -> 1 (default / collection)
df["loan_status_binary"] = (df["loan_status"] != "PAIDOFF").astype(int)

target_col = "loan_status_binary"
y = df[target_col]

Select numeric and categorical features, handle missing values, apply one-hot encoding to categorical variables, and standardize all features.

In [48]:
# 2. Select features & basic preprocessing

numeric_cols = ["Principal", "terms", "past_due_days", "age"]
categorical_cols = ["education", "Gender"]

X_raw = df[numeric_cols + categorical_cols].copy()

# Handle missing values:
#  - fill NaNs in 'past_due_days' with 0 to indicate "no overdue"
if "past_due_days" in X_raw.columns:
    X_raw["past_due_days"] = X_raw["past_due_days"].fillna(0)

X_num = X_raw[numeric_cols]
X_cat = X_raw[categorical_cols]

X_cat_encoded = pd.get_dummies(X_cat, drop_first=True)

X_processed = pd.concat([X_num, X_cat_encoded], axis=1)

# Standardize all features: mean 0, variance 1
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_processed)

Split the processed features and labels into training and validation sets, then convert them to PyTorch tensors and DataLoaders for batch training.

In [49]:
# 3. Train/validation split

X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Convert arrays to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_val = torch.tensor(y_val.values, dtype=torch.float32)

# Build DataLoaders for the training and validation sets
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

Define the MLP architecture for loan default prediction using only Linear layers, without inserting any nonlinear activation functions yet.

In [50]:
# 4. Define MLP model (NO activations between layers)

class LoanMLPLinear(nn.Module):
    def __init__(self, input_dim, hidden_dims=None):
        super().__init__()
        if hidden_dims is None:
            hidden_dims = [64, 32]

        layers = []
        in_dim = input_dim

        # Stack Linear layers without activations
        for h in hidden_dims:
            layers.append(nn.Linear(in_dim, h))
            # No activation function here on purpose
            in_dim = h

        # Output layer: single logit for binary classification
        layers.append(nn.Linear(in_dim, 1))

        self.net = nn.Sequential(*layers)

    def forward(self, x):
        x = self.net(x)
        # squeeze to shape [batch_size]
        return x.view(-1)

input_dim = X_train.shape[1]
model = LoanMLPLinear(input_dim=input_dim)

Set up the training configuration for the MLP model, including device, loss function (BCEWithLogitsLoss), optimizer (Adam), and dataset statistics.

In [51]:
# 5. Training setup

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Use BCEWithLogitsLoss which combines Sigmoid + BCELoss in a stable way
criterion = nn.BCEWithLogitsLoss()

# Adam optimizer with learning rate 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

print("Device:", device)
print("Input dim:", input_dim)
print("Training samples:", len(train_dataset))
print("Validation samples:", len(val_dataset))

Device: cpu
Input dim: 8
Training samples: 400
Validation samples: 100


Implement the training and evaluation loops for the MLP, and run a single epoch to ensure that the full loan default prediction pipeline works.

In [52]:
# 6. Training & evaluation loops

def train_one_epoch(model, loader, optimizer, criterion, device):
    # Train the model for one epoch on the given DataLoader.
    model.train()
    running_loss = 0.0
    for Xb, yb in loader:
        Xb, yb = Xb.to(device), yb.to(device)

        optimizer.zero_grad()
        logits = model(Xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * Xb.size(0)

    return running_loss / len(loader.dataset)


def evaluate(model, loader, device):
    # Evaluate the model on validation data (loss + accuracy).
    model.eval()
    total = 0
    correct = 0
    running_loss = 0.0
    with torch.no_grad():
        for Xb, yb in loader:
            Xb, yb = Xb.to(device), yb.to(device)
            logits = model(Xb)
            loss = criterion(logits, yb)

            probs = torch.sigmoid(logits)
            preds = (probs >= 0.5).float()

            correct += (preds == yb).sum().item()
            total += yb.size(0)
            running_loss += loss.item() * Xb.size(0)

    avg_loss = running_loss / total
    acc = correct / total
    return avg_loss, acc

num_epochs = 1

for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_loader, device)

    print(
        f"Epoch {epoch + 1}/{num_epochs} "
        f"- train_loss: {train_loss:.4f} "
        f"- val_loss: {val_loss:.4f} "
        f"- val_acc: {val_acc:.4f}"
    )

print("Initial MLP pipeline finished. You can now add activation functions and tune hyperparameters.")

Epoch 1/1 - train_loss: 0.6446 - val_loss: 0.5910 - val_acc: 0.8100
Initial MLP pipeline finished. You can now add activation functions and tune hyperparameters.
